In [0]:
!pip install -U -q PyDrive

In [0]:
import requests
import pandas as pd
import numpy as np
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
!mkdir data

In [0]:
file_id = '1XTHFtpUhkjC45L9Zg8Z5NN_SC46gWXf8'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('data/auto.json')

In [0]:
df = pd.read_json('data/auto.json', orient='records')

In [0]:
pd.options.display.float_format = '{:,.2f}'.format

In [8]:
df.head()

,CarNumber,Refund,Fines,Make,Model
0,Y163O8161RUS,2,"3,200.00",Ford,Focus
1,E432XX77RUS,1,"6,500.00",Toyota,Camry
2,7184TT36RUS,1,"2,100.00",Ford,Focus
3,X582HE161RUS,2,"2,000.00",Ford,Focus
4,92918M178RUS,1,"5,700.00",Ford,Focus


In [9]:
len(df)

825

In [0]:
np.random.seed(21)

In [0]:
indices = [np.random.randint(825) for i in range(200)]
data = {'CarNumber': [df['CarNumber'][i] for i in indices]}
data.update({col: [np.random.choice(df[col]) for i in range(200)]
             for col in ['Refund', 'Fines']})
data.update({col: [df[col][i] for i in indices] for col in ['Make', 'Model']})

df2 = pd.DataFrame(data)

In [18]:
df2.head()

,CarNumber,Refund,Fines,Make,Model
0,T020MM116RUS,2,"10,200.00",Skoda,Octavia
1,7361C8197RUS,2,"4,000.00",Ford,Focus
2,M580CH197RUS,2,"18,900.00",Ford,Focus
3,T940CC96RUS,2,"8,238.22",Ford,Focus
4,T6418M116RUS,1,"12,000.00",Ford,Focus


In [0]:
concat_rows = pd.concat([df, df2])

In [20]:
concat_rows.head()

,CarNumber,Refund,Fines,Make,Model
0,Y163O8161RUS,2,"3,200.00",Ford,Focus
1,E432XX77RUS,1,"6,500.00",Toyota,Camry
2,7184TT36RUS,1,"2,100.00",Ford,Focus
3,X582HE161RUS,2,"2,000.00",Ford,Focus
4,92918M178RUS,1,"5,700.00",Ford,Focus


In [21]:
concat_rows.count()

CarNumber    1025
Refund       1025
Fines        1025
Make         1025
Model        1006
dtype: int64

In [0]:
fines = concat_rows

In [0]:
fines['Year'] = pd.Series([np.random.randint(1980, 2020) for i in range(1025)])

In [24]:
fines.head()

,CarNumber,Refund,Fines,Make,Model,Year
0,Y163O8161RUS,2,"3,200.00",Ford,Focus,1983
1,E432XX77RUS,1,"6,500.00",Toyota,Camry,1996
2,7184TT36RUS,1,"2,100.00",Ford,Focus,1983
3,X582HE161RUS,2,"2,000.00",Ford,Focus,1997
4,92918M178RUS,1,"5,700.00",Ford,Focus,2007


In [25]:
fines.count()

CarNumber    1025
Refund       1025
Fines        1025
Make         1025
Model        1006
Year         1025
dtype: int64

In [0]:
url = 'https://api.census.gov/data/2010/surname?get=NAME,COUNT&RANK=1:100'
chars = '""[],0123456789'
page = requests.get(url).text
surnames = [line.strip(chars).capitalize() for line in page.split('\n')]

In [0]:
owners = pd.DataFrame(
    {'CarNumber': fines['CarNumber'][:825],
     'NAME': [np.random.choice(surnames) for i in range(825)]}
     )

In [28]:
owners.head()

,CarNumber,NAME
0,Y163O8161RUS,Patel
1,E432XX77RUS,Wright
2,7184TT36RUS,Rodriguez
3,X582HE161RUS,Young
4,92918M178RUS,Cox


In [0]:
digits = list('0123456789')
letters = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ')

In [0]:
def generate_car_number():
    res = np.random.choice(digits + letters)
    for i in range(3):
        res += np.random.choice(digits)
    for i in range(2):
        res += np.random.choice(digits + letters)
    for i in range(3):
        res += np.random.choice(digits)
    res += 'RUS'
    return res

In [0]:
data = {'CarNumber': [generate_car_number() for i in range(5)]}
data.update({col: [np.random.choice(fines[col]) for i in range(5)]
             for col in ['Refund', 'Fines', 'Make', 'Model', 'Year']})

fines = pd.concat([fines, pd.DataFrame(data)])

In [32]:
len(fines)

1030

In [0]:
owners = owners.drop(owners.tail(20).index)

In [0]:
data = {'CarNumber': [generate_car_number() for i in range(5)],
        'NAME': [np.random.choice(surnames) for i in range(5)]}

owners = pd.concat([owners, pd.DataFrame(data)])

In [35]:
len(owners)

810

In [36]:
pd.pivot_table(fines, values='Fines', index=['Make', 'Model'], columns=['Year'],
               aggfunc=np.sum)

Year                     1980       1981  ...      2018       2019
Make       Model                          ...                     
Ford       Focus   125,552.88 227,338.22  ... 79,700.00 171,076.44
           Mondeo         nan  19,800.00  ...       nan        nan
Skoda      Octavia  10,238.22        nan  ... 11,600.00  19,600.00
Toyota     Camry          nan  19,800.00  ...       nan   8,238.22
           Corolla   8,000.00  34,300.00  ...       nan        nan
Volkswagen Focus          nan        nan  ...       nan        nan
           Golf           nan  10,600.00  ...    200.00  18,400.00
           Jetta          nan        nan  ...  9,000.00        nan
           Passat         nan        nan  ...  8,100.00   9,000.00
           Touareg        nan        nan  ...       nan        nan
Volvo      Focus          nan        nan  ...       nan        nan

[11 rows x 40 columns]

In [0]:
fines.to_csv('data/fines.csv', index=False)
owners.to_csv('data/owners.csv', index=False)

In [0]:
uploaded = drive.CreateFile({'title': 'fines.csv'})
uploaded.SetContentFile('data/fines.csv')
uploaded.Upload()